In [1]:
# automatically reload solution_augmentation if it is altered and used in the
# notebook
%load_ext autoreload
%autoreload 2

%matplotlib widget


import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd

from lib_augmentation import *

data_path = os.path.expandvars('C:/Users/$USERNAME/Desktop/data/06_data_augmentation/')

path_clean_speech = os.path.join(data_path, 'test/clean/')
path_rirs = os.path.join(data_path, 'test/rir/')

clean_speech, _ = librosa.load(os.path.join(path_clean_speech, 'p227_024.wav'), sr=16E3)
clean_speech -= np.mean(clean_speech)

noise1, _ = librosa.load(os.path.join(data_path, 'test/noise/1-137-A-32.wav'), sr=16E3)
noise2, _ = librosa.load(os.path.join(data_path, 'test/noise/1-17092-A-27.wav'), sr=16E3)
noise = np.concatenate((noise1, noise2))
noise -= np.mean(noise)
noise = noise[:len(clean_speech)]

rir1, _ = librosa.load(os.path.join(path_rirs, '1221.wav'), sr=16E3)
rir2, _ = librosa.load(os.path.join(path_rirs, '1201.wav'), sr=16E3)

C:\Users\kaspe\AppData\Local\Temp\ipykernel_24356\691879.py:22: UserWarning: PySoundFile failed. Trying audioread instead.
  clean_speech, _ = librosa.load(os.path.join(path_clean_speech, 'p227_024.wav'), sr=16E3)
c:\Users\kaspe\OneDrive - Aalto University\RWTH\Courses\61.00004 Audio Processing Using Python\code prerequisites-20240421\venv\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/kaspe/Desktop/data/06_data_augmentation/test/clean/p227_024.wav'

## 6.1 Voice Activity Detection

##### Plot Histogram of Log Power

In [ ]:
# YOUR CODE HERE

##### Fit GMM and Show Mean & Variances

In [ ]:
# YOUR CODE HERE

##### Plot Gaussians in Histogram

In [ ]:
def gaussian(x, m, cov):
    """ Return probability for value(s) *x* of gaussian / normal distribution 
        with mean *m* and standard deviation *cov*
    """
    std = np.sqrt(cov)
    y = np.exp(-((x - m) / std)**2 / 2) / np.sqrt(2*np.pi) / std
    return y

# YOUR CODE HERE

##### Test Function `vad_extraction`

In [ ]:
vad = vad_extraction(clean_speech)

_, axis= plt.subplots(1)

axis.plot(clean_speech)

xs = np.arange(len(vad)) * 160
axis.plot(xs, vad * np.max(np.abs(vad)))

## 6.2 Test 'Mix Noise and Speech'

In [ ]:
clean_speech_changed, noisy_speech = mix(clean_speech, noise, 5, [rir1, rir2])

_, axis = plt.subplots(1)
axis.plot(noisy_speech)
axis.plot(clean_speech_changed)

In [ ]:
sr = 16000
sd.play(np.concatenate((clean_speech, noisy_speech)), samplerate=sr)

## 6.3 Test 'Feature Extraction'

In [ ]:
features = feature_extraction(noisy_speech)
print(features.shape, '\n\n', features)

## 6.4 Test 'Generators'

In [ ]:
# test init, i.e. creating generator
gen_speech = GenSpeech(path_clean_speech)

# test __next__() two times
clean_speech_1 = next(gen_speech)
clean_speech_2 = next(gen_speech)

# plot retrieved example speech files
_, axes = plt.subplots(1,2)
axes[0].plot(clean_speech_1)
axes[1].plot(clean_speech_2)

# test __len__()
print(len(gen_speech))

# test if the generator runs until completion & stops
for speech in gen_speech:
    pass

In [ ]:
# test init, i.e. creating generator
gen_rirs = create_gen_rir(path_rirs)

# test __next__() two times
rir1 = next(gen_rirs)
rir2 = next(gen_rirs)

# plot retrieved example rirs
_, axes = plt.subplots(1,2)
axes[0].plot(rir1)
axes[1].plot(rir2)

## 6.5 Run Augmentation

In [ ]:
for split in ['train', 'val', 'test']:
    path = os.path.join(data_path, split)
    run_augmentation(path, debug=True)